# Sales Tax Data

In [71]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from datetime import date
import cpi

#pd.set_option('display.max_rows', 1000) 
#pd.set_option('display.max_columns', 1000)
#pd.set_option("display.max_colwidth", None) 

In [34]:
cpi.update()

In [2]:
blocks = pd.read_csv("raw_data/census_blocks_2020.csv", dtype={"GEOID20": "string"})
blocks.head()

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,data_as_of,data_loaded_at,multipolygon
0,6,75,10101,1001,60750101011001,Block 1001,G5040,S,"262,902",0,37.808484,-122.409904,0,15,2022 Jul 21 10:57:39 PM,2022 Jul 25 01:59:00 PM,"MULTIPOLYGON (((-122.420353 37.81151, -122.420..."
1,6,75,47801,1005,60750478011005,Block 1005,G5040,S,"19,608",0,37.773038,-122.494435,71,173,2022 Jul 21 10:57:39 PM,2022 Jul 25 01:59:00 PM,"MULTIPOLYGON (((-122.495039 37.773947, -122.49..."
2,6,75,15401,1005,60750154011005,Block 1005,G5040,S,"14,331",0,37.783112,-122.448710,44,150,2022 Jul 21 10:57:39 PM,2022 Jul 25 01:59:00 PM,"MULTIPOLYGON (((-122.449358 37.783692, -122.44..."
3,6,75,32901,2003,60750329012003,Block 2003,G5040,S,"19,646",0,37.747121,-122.489407,55,168,2022 Jul 21 10:57:39 PM,2022 Jul 25 01:59:00 PM,"MULTIPOLYGON (((-122.490009 37.748031, -122.48..."
4,6,75,15401,2002,60750154012002,Block 2002,G5040,S,"26,899",0,37.782709,-122.456171,62,178,2022 Jul 21 10:57:39 PM,2022 Jul 25 01:59:00 PM,"MULTIPOLYGON (((-122.456813 37.783929, -122.45..."


In [3]:
# turn into shapely object
blocks["geometry"] = blocks["multipolygon"].apply(wkt.loads)

In [4]:
#geodataframe for geopandas
# we want a GDF with really just the census block IDs and the shapely geometry objects

gdf_blocks = gpd.GeoDataFrame(blocks, geometry="geometry", crs="EPSG:4326")[["GEOID20", "geometry"]]
gdf_blocks.head()

,GEOID20,geometry
0,60750101011001,"MULTIPOLYGON (((-122.42035 37.81151, -122.42 3..."
1,60750478011005,"MULTIPOLYGON (((-122.49504 37.77395, -122.4939..."
2,60750154011005,"MULTIPOLYGON (((-122.44936 37.78369, -122.4483..."
3,60750329012003,"MULTIPOLYGON (((-122.49001 37.74803, -122.4889..."
4,60750154012002,"MULTIPOLYGON (((-122.45681 37.78393, -122.4557..."


In [5]:
sales_tax = pd.read_csv("raw_data/sales-tax.csv")
sales_tax.head()

,quarter,area,amount
0,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44"
1,2025Q2,60750101011015 | 60750101012002 | 607501010120...,"29,328.64"
2,2025Q2,60750101011016 | 60750101012000 | 607501020210...,"61,111.45"
3,2025Q2,60750101011020 | 60750101021001 | 607501010210...,"41,524.44"
4,2025Q2,60750101012001 | 60750101012007 | 607501010120...,"43,155.05"


In [6]:
# convert sales tax to int at some point
sales_tax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44042 entries, 0 to 44041
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   quarter  44042 non-null  object
 1   area     44042 non-null  object
 2   amount   44042 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


In [7]:
# we need a unique identifier for each year + quarter 

sales_tax["group_id"] = sales_tax.index
sales_tax.head()

,quarter,area,amount,group_id
0,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0
1,2025Q2,60750101011015 | 60750101012002 | 607501010120...,"29,328.64",1
2,2025Q2,60750101011016 | 60750101012000 | 607501020210...,"61,111.45",2
3,2025Q2,60750101011020 | 60750101021001 | 607501010210...,"41,524.44",3
4,2025Q2,60750101012001 | 60750101012007 | 607501010120...,"43,155.05",4


In [8]:
# clean up the area column

sales_tax["block_list"] = sales_tax["area"].str.replace(" ", "", regex = False)
sales_tax["block_list"] = sales_tax["block_list"].str.split("|")
sales_tax.head()

,quarter,area,amount,group_id,block_list
0,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,"[60750101011005, 60750105001002, 6075010500102..."
1,2025Q2,60750101011015 | 60750101012002 | 607501010120...,"29,328.64",1,"[60750101011015, 60750101012002, 6075010101200..."
2,2025Q2,60750101011016 | 60750101012000 | 607501020210...,"61,111.45",2,"[60750101011016, 60750101012000, 6075010202100..."
3,2025Q2,60750101011020 | 60750101021001 | 607501010210...,"41,524.44",3,"[60750101011020, 60750101021001, 6075010102100..."
4,2025Q2,60750101012001 | 60750101012007 | 607501010120...,"43,155.05",4,"[60750101012001, 60750101012007, 6075010101200..."


In [9]:
# separate the list and rename for merge

sales_tax_long = sales_tax.explode("block_list", ignore_index=True)
sales_tax_long = sales_tax_long.rename(columns={"block_list": "GEOID20"})
sales_tax_long["GEOID20"] = sales_tax_long["GEOID20"].astype("string")
sales_tax_long.head()

,quarter,area,amount,group_id,GEOID20
0,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011005
1,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750105001002
2,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750105001021
3,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011000
4,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011007


- we added a unique identifier to each year+ quarter
- we split the census blocks to be grouped into different rows
- we renamed the geometry merging column

### Join geometries

- merge sales tax data with shapely object geometry 
- convert to geodataframe 
- group the geometries together for each unique group_id
- retain the sales tax number

In [10]:
gdf_sales = sales_tax_long.merge(gdf_blocks, on = "GEOID20", how = 'left')

gdf_sales = gpd.GeoDataFrame(gdf_sales, geometry="geometry", crs="EPSG:4326")

gdf_sales.head()

,quarter,area,amount,group_id,GEOID20,geometry
0,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011005,"MULTIPOLYGON (((-122.41584 37.80897, -122.4155..."
1,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750105001002,"MULTIPOLYGON (((-122.4035 37.80509, -122.40234..."
2,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750105001021,"MULTIPOLYGON (((-122.40499 37.80383, -122.4033..."
3,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011000,"MULTIPOLYGON (((-122.42108 37.81289, -122.4201..."
4,2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",0,60750101011007,"MULTIPOLYGON (((-122.40905 37.80807, -122.4074..."


In [11]:
gdf_sales["geometry"].isna().sum()

0

In [12]:
# dissolve to get one reshaped polygon per row. 

gdf_groups = gdf_sales.dissolve(by="group_id", aggfunc="first").reset_index()

gdf_groups.head()

,group_id,geometry,quarter,area,amount,GEOID20
0,0,"MULTIPOLYGON (((-122.40316 37.8031, -122.40352...",2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",60750101011005
1,1,"MULTIPOLYGON (((-122.39848 37.80723, -122.3933...",2025Q2,60750101011015 | 60750101012002 | 607501010120...,"29,328.64",60750101011015
2,2,"MULTIPOLYGON (((-122.40499 37.80383, -122.4053...",2025Q2,60750101011016 | 60750101012000 | 607501020210...,"61,111.45",60750101011016
3,3,"MULTIPOLYGON (((-122.41322 37.80376, -122.4137...",2025Q2,60750101011020 | 60750101021001 | 607501010210...,"41,524.44",60750101011020
4,4,"MULTIPOLYGON (((-122.42347 37.80436, -122.4250...",2025Q2,60750101012001 | 60750101012007 | 607501010120...,"43,155.05",60750101012001


#### Tagging Neighborhoods

In [13]:
an_df = pd.read_csv("raw_data/analysis_neighborhood.csv", dtype={"geoid": "string"})[["geoid", "neighborhoods_analysis_boundaries"]]
an_df.head()

,geoid,neighborhoods_analysis_boundaries
0,06075980900,Bayview Hunters Point
1,06075980600,Bayview Hunters Point
2,06075980501,McLaren Park
3,06075980401,The Farallones
4,06075061200,Bayview Hunters Point


In [14]:
target_neighborhoods = [
    "Mission",
    "Tenderloin",
    "Inner Richmond",
    "Outer Richmond",
    "Sunset/Parkside",
    "Inner Sunset",
    "Chinatown",
    "Bayview Hunters Point",
]

an_six = an_df[an_df["neighborhoods_analysis_boundaries"].isin(target_neighborhoods)].copy()

an_six.head()
len(an_six)

84

In [15]:
# sanity checks
len(gdf_groups)

44042

In [16]:
len(sales_tax)

44042

In [17]:
# create a look up dictionary

tract_lookup = dict(zip(
    an_six["geoid"],
    an_six["neighborhoods_analysis_boundaries"]
))

In [18]:
len(tract_lookup)

84

In [19]:
def classify_group(area_str):
    if pd.isna(area_str):
        return None
    
    # split | and remove spaces
    blocks = area_str.replace(" ", "").split("|")

    # loop through block IDs 
    for block in blocks:
        
        tract = "06075" + block[4:10]
        if tract in tract_lookup:
            return tract_lookup[tract]

    return None

In [20]:
gdf_groups["analysis_neighborhood"] = gdf_groups["area"].apply(classify_group)
gdf_groups.head()

,group_id,geometry,quarter,area,amount,GEOID20,analysis_neighborhood
0,0,"MULTIPOLYGON (((-122.40316 37.8031, -122.40352...",2025Q2,60750101011005 | 60750105001002 | 607501050010...,"118,140.44",60750101011005,None
1,1,"MULTIPOLYGON (((-122.39848 37.80723, -122.3933...",2025Q2,60750101011015 | 60750101012002 | 607501010120...,"29,328.64",60750101011015,None
2,2,"MULTIPOLYGON (((-122.40499 37.80383, -122.4053...",2025Q2,60750101011016 | 60750101012000 | 607501020210...,"61,111.45",60750101011016,None
3,3,"MULTIPOLYGON (((-122.41322 37.80376, -122.4137...",2025Q2,60750101011020 | 60750101021001 | 607501010210...,"41,524.44",60750101011020,None
4,4,"MULTIPOLYGON (((-122.42347 37.80436, -122.4250...",2025Q2,60750101012001 | 60750101012007 | 607501010120...,"43,155.05",60750101012001,None


In [21]:
gdf_groups["analysis_neighborhood"].value_counts(dropna=False)

analysis_neighborhood
None                     25522
Sunset/Parkside           4030
Mission                   3637
Bayview Hunters Point     2820
Outer Richmond            1949
Inner Sunset              1827
Tenderloin                1667
Chinatown                 1385
Inner Richmond            1205
Name: count, dtype: int64

In [22]:
gdf_groups["amount"] = gdf_groups["amount"].str.replace(",","").astype(float)
gdf_groups.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 44042 entries, 0 to 44041
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   group_id               44042 non-null  int64   
 1   geometry               44042 non-null  geometry
 2   quarter                44042 non-null  object  
 3   area                   44042 non-null  object  
 4   amount                 44042 non-null  float64 
 5   GEOID20                44042 non-null  string  
 6   analysis_neighborhood  18520 non-null  object  
dtypes: float64(1), geometry(1), int64(1), object(3), string(1)
memory usage: 2.4+ MB


In [23]:
# export to geojson to use in Mapbox
gdf_groups = gdf_groups.to_crs(4326)
gdf_groups.to_file("data_with_geometry/sf_sales_tax_polygons.geojson", driver="GeoJSON")

In [24]:
# Also save a version with geometry for computation

gdf_groups_wkt = gdf_groups.copy()
gdf_groups_wkt["geometry"] = gdf_groups_wkt["geometry"].astype(str)

gdf_groups_wkt.to_csv("data_with_geometry/sf_sales_tax_groups_with_geometry.csv", index=False)

/var/folders/2_/nl4h85lj1jqcl64vgwgvqnh40000gn/T/ipykernel_67081/3115724397.py:4: UserWarning: Geometry column does not contain geometry.
  gdf_groups_wkt["geometry"] = gdf_groups_wkt["geometry"].astype(str)


# Calculating Neighborhood Recovery

In [30]:
sales_tax = pd.read_csv("data_with_geometry/sf_sales_tax_groups_with_geometry.csv")
sales_tax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44042 entries, 0 to 44041
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   group_id               44042 non-null  int64  
 1   geometry               44042 non-null  object 
 2   quarter                44042 non-null  object 
 3   area                   44042 non-null  object 
 4   amount                 44042 non-null  float64
 5   GEOID20                44042 non-null  int64  
 6   analysis_neighborhood  18520 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 2.4+ MB


#### Lets do it for Mission first and then we can run it for all

To be clear, we are first comparing Q2 2019 with Q2 2025

First we need to adjust for inflation

In [36]:
mission_recovery = sales_tax[(sales_tax["analysis_neighborhood"] == "Mission") & sales_tax["quarter"].isin(["2025Q2","2019Q2"])]

In [37]:
mission_recovery.head()

,group_id,geometry,quarter,area,amount,GEOID20,analysis_neighborhood
305,305,"MULTIPOLYGON (((-122.41761 37.77042, -122.4175...",2025Q2,60750176034000 | 60750177002009 | 607502010110...,371415.73,60750176034000,Mission
308,308,"MULTIPOLYGON (((-122.40726 37.76928, -122.4068...",2025Q2,60750177001001 | 60750180003001 | 607501800040...,40642.25,60750177001001,Mission
309,309,"POLYGON ((-122.40768 37.76708, -122.40753 37.7...",2025Q2,60750177001006 | 60750177001015 | 607501770010...,132062.48,60750177001006,Mission
310,310,"MULTIPOLYGON (((-122.41049 37.76561, -122.4114...",2025Q2,60750177001009 | 60750177001028 | 607501770010...,14989.88,60750177001009,Mission
311,311,"MULTIPOLYGON (((-122.41158 37.76684, -122.4114...",2025Q2,60750177001011 | 60750177002013 | 607501780320...,229689.50,60750177001011,Mission


In [64]:
mission_recovery = mission_recovery.pivot_table(index = ["geometry", "area",],
                      columns = "quarter",
                      values = "amount",
                      aggfunc = "sum").reset_index()

In [69]:
mission_recovery.head(2)

quarter,geometry,area,2019Q2,2025Q2
0,"MULTIPOLYGON (((-122.41778 37.73918, -122.4179 37.73842, -122.41907 37.73854, -122.41858 37.7393, -122.41846 37.73925, -122.41778 37.73918)), ((-122.42028 37.73998, -122.42086 37.74022, -122.42038 37.74096, -122.42005 37.74146, -122.41947 37.74123, -122.42028 37.73998)), ((-122.41579 37.74041, -122.41569 37.74161, -122.41567 37.74195, -122.41565 37.74213, -122.41522 37.74216, -122.41501 37.74218, -122.41507 37.74158, -122.41517 37.74038, -122.41579 37.74041)), ((-122.42017 37.74142, -122.42098 37.74175, -122.42069 37.74219, -122.41981 37.74183, -122.42005 37.74146, -122.42017 37.74142)), ((-122.41836 37.7482, -122.41821 37.7482, -122.41822 37.74817, -122.41826 37.74811, -122.41828 37.74808, -122.41912 37.74678, -122.4195 37.74618, -122.41989 37.7456, -122.42002 37.74539, -122.42013 37.74658, -122.42027 37.74803, -122.42027 37.74806, -122.42028 37.74812, -122.42028 37.74816, -122.42014 37.74816, -122.41917 37.74818, -122.41836 37.7482)), ((-122.42256 37.748, -122.42288 37.74795, -122.42458 37.74785, -122.4247 37.74784, -122.42478 37.74864, -122.42368 37.74871, -122.42261 37.74878, -122.42256 37.748)))",60750253004000 | 60750254012002 | 60750210004004 | 60750252004001 | 60750252004005 | 60750253002003 | 60750253002005,40700.69,21689.80
1,"MULTIPOLYGON (((-122.41382 37.74829, -122.41367 37.7483, -122.41367 37.74826, -122.41367 37.74822, -122.41366 37.74816, -122.41357 37.74717, -122.4137 37.7472, -122.41483 37.74734, -122.41491 37.74815, -122.41491 37.74817, -122.41492 37.74823, -122.41492 37.74828, -122.41483 37.74827, -122.41382 37.74829)), ((-122.41702 37.74877, -122.41736 37.74853, -122.41791 37.74832, -122.41799 37.74832, -122.41821 37.7482, -122.41836 37.7482, -122.41917 37.74818, -122.41925 37.74897, -122.41871 37.74901, -122.41814 37.74904, -122.41829 37.75064, -122.41773 37.75067, -122.4172 37.7507, -122.41705 37.74911, -122.41702 37.74877)), ((-122.41485 37.74902, -122.41485 37.74892, -122.41485 37.74891, -122.41486 37.74888, -122.41487 37.74886, -122.4149 37.74883, -122.41493 37.74882, -122.41497 37.7488, -122.41498 37.74879, -122.41499 37.74878, -122.41499 37.74875, -122.41499 37.7487, -122.41499 37.74868, -122.41498 37.74864, -122.41492 37.74828, -122.41587 37.74827, -122.41587 37.7483, -122.41596 37.74917, -122.41539 37.74921, -122.41486 37.74924, -122.41485 37.74902)), ((-122.41652 37.74914, -122.41667 37.75074, -122.41611 37.75077, -122.41596 37.74917, -122.41652 37.74914)))",60750229013004 | 60750252001002 | 60750209002000 | 60750209002004 | 60750209002002 | 60750209002003 | 60750209003005,19484.90,22035.41


In [89]:
# adjusting for inflation


mission_recovery["CPI_adjusted_2025"] = mission_recovery["2025Q2"].apply(lambda v: cpi.inflate(v,date(2025, 6, 30),to=date(2019, 6, 30)))
mission_recovery.head()

quarter,geometry,area,2019Q2,2025Q2,CPI_adjusted_2025
0,"MULTIPOLYGON (((-122.41778 37.73918, -122.4179 37.73842, -122.41907 37.73854, -122.41858 37.7393, -122.41846 37.73925, -122.41778 37.73918)), ((-122.42028 37.73998, -122.42086 37.74022, -122.42038 37.74096, -122.42005 37.74146, -122.41947 37.74123, -122.42028 37.73998)), ((-122.41579 37.74041, -122.41569 37.74161, -122.41567 37.74195, -122.41565 37.74213, -122.41522 37.74216, -122.41501 37.74218, -122.41507 37.74158, -122.41517 37.74038, -122.41579 37.74041)), ((-122.42017 37.74142, -122.42098 37.74175, -122.42069 37.74219, -122.41981 37.74183, -122.42005 37.74146, -122.42017 37.74142)), ((-122.41836 37.7482, -122.41821 37.7482, -122.41822 37.74817, -122.41826 37.74811, -122.41828 37.74808, -122.41912 37.74678, -122.4195 37.74618, -122.41989 37.7456, -122.42002 37.74539, -122.42013 37.74658, -122.42027 37.74803, -122.42027 37.74806, -122.42028 37.74812, -122.42028 37.74816, -122.42014 37.74816, -122.41917 37.74818, -122.41836 37.7482)), ((-122.42256 37.748, -122.42288 37.74795, -122.42458 37.74785, -122.4247 37.74784, -122.42478 37.74864, -122.42368 37.74871, -122.42261 37.74878, -122.42256 37.748)))",60750253004000 | 60750254012002 | 60750210004004 | 60750252004001 | 60750252004005 | 60750253002003 | 60750253002005,40700.69,21689.80,17223.689291
1,"MULTIPOLYGON (((-122.41382 37.74829, -122.41367 37.7483, -122.41367 37.74826, -122.41367 37.74822, -122.41366 37.74816, -122.41357 37.74717, -122.4137 37.7472, -122.41483 37.74734, -122.41491 37.74815, -122.41491 37.74817, -122.41492 37.74823, -122.41492 37.74828, -122.41483 37.74827, -122.41382 37.74829)), ((-122.41702 37.74877, -122.41736 37.74853, -122.41791 37.74832, -122.41799 37.74832, -122.41821 37.7482, -122.41836 37.7482, -122.41917 37.74818, -122.41925 37.74897, -122.41871 37.74901, -122.41814 37.74904, -122.41829 37.75064, -122.41773 37.75067, -122.4172 37.7507, -122.41705 37.74911, -122.41702 37.74877)), ((-122.41485 37.74902, -122.41485 37.74892, -122.41485 37.74891, -122.41486 37.74888, -122.41487 37.74886, -122.4149 37.74883, -122.41493 37.74882, -122.41497 37.7488, -122.41498 37.74879, -122.41499 37.74878, -122.41499 37.74875, -122.41499 37.7487, -122.41499 37.74868, -122.41498 37.74864, -122.41492 37.74828, -122.41587 37.74827, -122.41587 37.7483, -122.41596 37.74917, -122.41539 37.74921, -122.41486 37.74924, -122.41485 37.74902)), ((-122.41652 37.74914, -122.41667 37.75074, -122.41611 37.75077, -122.41596 37.74917, -122.41652 37.74914)))",60750229013004 | 60750252001002 | 60750209002000 | 60750209002004 | 60750209002002 | 60750209002003 | 60750209003005,19484.90,22035.41,17498.135310
2,"POLYGON ((-122.41539 37.74921, -122.41596 37.74917, -122.41611 37.75077, -122.41667 37.75074, -122.4172 37.7507, -122.41736 37.75231, -122.41683 37.75234, -122.41627 37.75237, -122.41517 37.75244, -122.41501 37.75084, -122.41554 37.7508, -122.41539 37.74921))",60750229011004 | 60750229013003 | 60750209001005 | 60750209001004,12406.32,8568.39,6804.087040
3,"MULTIPOLYGON (((-122.41736 37.74853, -122.41702 37.74877, -122.41705 37.74911, -122.41652 37.74914, -122.41596 37.74917, -122.41587 37.7483, -122.41587 37.74827, -122.41821 37.7482, -122.41799 37.74832, -122.41791 37.74832, -122.41736 37.74853)), ((-122.42083 37.74888, -122.42128 37.74885, -122.42139 37.74996, -122.42144 37.75045, -122.42098 37.75048, -122.42083 37.74888)), ((-122.41978 37.74894, -122.42035 37.74891, -122.42051 37.75051, -122.41993 37.75054, -122.41978 37.74894)), ((-122.41814 37.74904, -122.41871 37.74901, -122.41886 37.7506, -122.41829 37.75064, -122.41814 37.74904)))",60750209002005 | 60750209003000 | 60750209003003 | 60750210004001,38972.81,36643.85,29098.575682
4,"MULTIPOLYGON (((-122.42128 37.74885, -122.42083 37.74888, -122.42035 37.74891, -122.41978 37.74894, -122.41925 37.74897, -122.41917 37.74818, -122.42014 37.74816, -122.42028 37.74816, -122.42139 37.74812, -122.4223 37.74807, -122.42246 37.74801, -122.42256 37.748, -122.42261 37.74878, -122.42

In [92]:
mission_recovery["pct_change"] = ((mission_recovery["CPI_adjusted_2025"] - mission_recovery["2019Q2"])/ mission_recovery["2019Q2"]) * 100
mission_recovery.head()

quarter,geometry,area,2019Q2,2025Q2,CPI_adjusted_2025,pct_change
0,"MULTIPOLYGON (((-122.41778 37.73918, -122.4179 37.73842, -122.41907 37.73854, -122.41858 37.7393, -122.41846 37.73925, -122.41778 37.73918)), ((-122.42028 37.73998, -122.42086 37.74022, -122.42038 37.74096, -122.42005 37.74146, -122.41947 37.74123, -122.42028 37.73998)), ((-122.41579 37.74041, -122.41569 37.74161, -122.41567 37.74195, -122.41565 37.74213, -122.41522 37.74216, -122.41501 37.74218, -122.41507 37.74158, -122.41517 37.74038, -122.41579 37.74041)), ((-122.42017 37.74142, -122.42098 37.74175, -122.42069 37.74219, -122.41981 37.74183, -122.42005 37.74146, -122.42017 37.74142)), ((-122.41836 37.7482, -122.41821 37.7482, -122.41822 37.74817, -122.41826 37.74811, -122.41828 37.74808, -122.41912 37.74678, -122.4195 37.74618, -122.41989 37.7456, -122.42002 37.74539, -122.42013 37.74658, -122.42027 37.74803, -122.42027 37.74806, -122.42028 37.74812, -122.42028 37.74816, -122.42014 37.74816, -122.41917 37.74818, -122.41836 37.7482)), ((-122.42256 37.748, -122.42288 37.74795, -122.42458 37.74785, -122.4247 37.74784, -122.42478 37.74864, -122.42368 37.74871, -122.42261 37.74878, -122.42256 37.748)))",60750253004000 | 60750254012002 | 60750210004004 | 60750252004001 | 60750252004005 | 60750253002003 | 60750253002005,40700.69,21689.80,17223.689291,-57.682071
1,"MULTIPOLYGON (((-122.41382 37.74829, -122.41367 37.7483, -122.41367 37.74826, -122.41367 37.74822, -122.41366 37.74816, -122.41357 37.74717, -122.4137 37.7472, -122.41483 37.74734, -122.41491 37.74815, -122.41491 37.74817, -122.41492 37.74823, -122.41492 37.74828, -122.41483 37.74827, -122.41382 37.74829)), ((-122.41702 37.74877, -122.41736 37.74853, -122.41791 37.74832, -122.41799 37.74832, -122.41821 37.7482, -122.41836 37.7482, -122.41917 37.74818, -122.41925 37.74897, -122.41871 37.74901, -122.41814 37.74904, -122.41829 37.75064, -122.41773 37.75067, -122.4172 37.7507, -122.41705 37.74911, -122.41702 37.74877)), ((-122.41485 37.74902, -122.41485 37.74892, -122.41485 37.74891, -122.41486 37.74888, -122.41487 37.74886, -122.4149 37.74883, -122.41493 37.74882, -122.41497 37.7488, -122.41498 37.74879, -122.41499 37.74878, -122.41499 37.74875, -122.41499 37.7487, -122.41499 37.74868, -122.41498 37.74864, -122.41492 37.74828, -122.41587 37.74827, -122.41587 37.7483, -122.41596 37.74917, -122.41539 37.74921, -122.41486 37.74924, -122.41485 37.74902)), ((-122.41652 37.74914, -122.41667 37.75074, -122.41611 37.75077, -122.41596 37.74917, -122.41652 37.74914)))",60750229013004 | 60750252001002 | 60750209002000 | 60750209002004 | 60750209002002 | 60750209002003 | 60750209003005,19484.90,22035.41,17498.135310,-10.196433
2,"POLYGON ((-122.41539 37.74921, -122.41596 37.74917, -122.41611 37.75077, -122.41667 37.75074, -122.4172 37.7507, -122.41736 37.75231, -122.41683 37.75234, -122.41627 37.75237, -122.41517 37.75244, -122.41501 37.75084, -122.41554 37.7508, -122.41539 37.74921))",60750229011004 | 60750229013003 | 60750209001005 | 60750209001004,12406.32,8568.39,6804.087040,-45.156283
3,"MULTIPOLYGON (((-122.41736 37.74853, -122.41702 37.74877, -122.41705 37.74911, -122.41652 37.74914, -122.41596 37.74917, -122.41587 37.7483, -122.41587 37.74827, -122.41821 37.7482, -122.41799 37.74832, -122.41791 37.74832, -122.41736 37.74853)), ((-122.42083 37.74888, -122.42128 37.74885, -122.42139 37.74996, -122.42144 37.75045, -122.42098 37.75048, -122.42083 37.74888)), ((-122.41978 37.74894, -122.42035 37.74891, -122.42051 37.75051, -122.41993 37.75054, -122.41978 37.74894)), ((-122.41814 37.74904, -122.41871 37.74901, -122.41886 37.7506, -122.41829 37.75064, -122.41814 37.74904)))",60750209002005 | 60750209003000 | 60750209003003 | 60750210004001,38972.81,36643.85,29098.575682,-25.336213
4,"MULTIPOLYGON (((-122.42128 37.74885, -122.42083 37.74888, -122.42035 37.74891, -122.41978 37.74894, -122.41925 37.74897, -122.41917 37.74818, -122.42014 37.74816, -122.42028 37.74816, -122.42139 37.74812, -122.4223 37.74807, -122.42246 37

In [96]:
mission_recovery = gpd.GeoDataFrame(mission_recovery, geometry="geometry", crs="EPSG:4326")

In [98]:
mission_recovery.to_file("docs/Mission/mission_recovery.geojson",driver="GeoJSON")

In [101]:
import json

with open("docs/Mission/mission_recovery.geojson") as f:
    data = json.load(f)

# show the first feature cleanly
data["features"][0]


{'type': 'Feature',
 'properties': {'area': '60750253004000 | 60750254012002 | 60750210004004 | 60750252004001 | 60750252004005 | 60750253002003 | 60750253002005',
  '2019Q2': 40700.69,
  '2025Q2': 21689.8,
  'CPI_adjusted_2025': 17223.68929101782,
  'pct_change': -57.68207052259354},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-122.417779, 37.739184],
     [-122.417896, 37.738424],
     [-122.419073, 37.738538],
     [-122.418584, 37.739297],
     [-122.418461, 37.739249],
     [-122.417779, 37.739184]]],
   [[[-122.420275, 37.739985],
     [-122.420856, 37.740221],
     [-122.420382, 37.740956],
     [-122.420051, 37.741465],
     [-122.419467, 37.741233],
     [-122.420275, 37.739985]]],
   [[[-122.415788, 37.740407],
     [-122.415692, 37.741609],
     [-122.415666, 37.741948],
     [-122.415651, 37.742133],
     [-122.415223, 37.742157],
     [-122.415014, 37.742182],
     [-122.415074, 37.74158],
     [-122.415169, 37.740377],
     [-122.415788, 37.740407]]],
   [[